### Imports

In [ ]:
from typing import Tuple
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader

import aim

In [ ]:
import os, sys
sys.path.append(os.path.abspath('..'))

from src.models.unet import UNet
from src.training.metrics import dice_loss
from src.training.metrics import evaluate
from src.training.train import train
from src.data.datasets import ACDCDataset


### Fetch dataset

In [ ]:

dataset = ACDCDataset(path='../../training/', tagged=False, verbose=1)

In [ ]:
train_set, val_set = random_split(dataset, [704, 248], generator=torch.Generator().manual_seed(42))
loader_train = DataLoader(train_set, batch_size=16, shuffle=True)
loader_val = DataLoader(val_set, batch_size=32, shuffle=False)

### Instantiate model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = UNet(n_channels=1, n_classes=4, bilinear=True).double()
saved_model = torch.load('../checkpoints/model/model_cine_v0.pt')
if isinstance(saved_model, nn.DataParallel):  # Extract UNet if saved model is parallelized
    saved_model = saved_model.module
model.load_state_dict(saved_model.state_dict())

if device.type == 'cuda':
    model = nn.DataParallel(model).to(device)
    model.n_classes = model.module.n_classes

### Train model

In [ ]:
# train(model, run=aim.Run(), loader_train=loader_train, loader_val=loader_val, device=device)

### Evaluate model outputs on validation set

In [ ]:
example, label = next(iter(loader_val))
out = model(example.double().to(device))

example.shape, label.shape, out.shape

In [ ]:
predicted_mask = F.softmax(out, dim=1).argmax(dim=1).cpu().detach().numpy().copy()

fig, ax = plt.subplots(4, 5, figsize=(10, 10))

for i in range(12, 16):
    for j in range(4):
        ax[i % 4, j].imshow(out.cpu().detach().numpy()[i, j])
        ax[i % 4, j].axis('off')

    ax[i % 4, 4].imshow(predicted_mask[i]), ax[i % 4, 4].axis('off')

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=(20, 15))

for i in range(8, 12):
    ax[0, i % 4].imshow(example[i, 0], cmap='gray')
    ax[1, i % 4].imshow(label[i], alpha=1)
    ax[2, i % 4].imshow(predicted_mask[i], alpha=1.)
    ax[0, i % 4].axis('off'), ax[1, i % 4].axis('off'), ax[2, i % 4].axis('off')